In [123]:
!pip install -Uqq fastai fastbook

In [124]:
import pandas as pd
import numpy as np

In [125]:
import torch 

# Check if GPU is available
print(torch.cuda.is_available())

True


In [126]:
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from fastai.data.core import DataLoaders, CrossEntropyLossFlat
from fastai.data.block import CategoryBlock
from fastai.basics import *

In [127]:
# Load a small chunk of the full dataset for testing purposes
faces = pd.read_csv('./faces_smaller_chunk.csv')
features = pd.read_csv('./vgg_chunks/chunk_1.csv', usecols=range(1, 2049))

In [128]:
faces.shape

(10859, 43)

In [129]:
features.shape

(10859, 2048)

In [130]:
features_tensor = torch.tensor(features.values).float()

In [131]:
features_tensor.shape

torch.Size([10859, 2048])

In [132]:
ground_truth = ['pol_dat_us', 'pol_dat_ca', 'pol_dat_uk', 'pol_fb_us']

In [133]:
def label_func(row):
    '''
    Checks each column of ground_truth and extracts whichever column is not null as the label
    '''
    for i in ground_truth:
        if ~np.isnan(row[i]):
            return row[i]
    return np.nan

def get_labels(data):
    '''
    Returns array of labels from entire dataset
    '''
    return data.apply(lambda row: label_func(row), axis=1).to_numpy()

get_labels(faces.iloc[[1,2,4,5]])

array([0., 1., 0., 0.])

In [134]:
# Randomly split training and testing datasets
num_of_samples = features_tensor.shape[0]
idx = np.random.permutation(range(num_of_samples))

cut = int(0.8 * num_of_samples)
train_idx = idx[:cut]
train_x = features_tensor[train_idx]
train_y = get_labels(faces.iloc[train_idx])

valid_idx = idx[cut:]
valid_x = features_tensor[valid_idx]
valid_y = get_labels(faces.iloc[valid_idx])

In [135]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data,self.labels = data,labels
        
    def __getitem__(self, i):
        sample = self.data[i]
        label = self.labels[i]
        return sample,label
    
    def __len__(self): return len(self.data)

train_ds = CustomDataset(train_x, train_y)
valid_ds = CustomDataset(valid_x, valid_y)

In [136]:
# Dataloader will be passing a batch of 64 samples at a time to our model
train_dl = DataLoader(train_ds, batch_size=64)
valid_dl = DataLoader(valid_ds, batch_size=64)

In [137]:
# Define a simple binary classifier that takes a 2048 feature long tensor as input
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()        
        
        # Number of input features is 2048
        self.layer_1 = nn.Linear(2048, 1024) 
        self.layer_2 = nn.Linear(1024, 480)
        self.layer_3 = nn.Linear(480, 240)
        self.layer_4 = nn.Linear(240, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(1024)
        self.batchnorm2 = nn.BatchNorm1d(480)
        self.batchnorm3 = nn.BatchNorm1d(240)
        self.batchnorm4 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.layer_out(x)
        
        return x

In [138]:
LEARNING_RATE = 0.003
EPOCHS = 50

In [139]:
import torch.optim as optim

model = BinaryClassifier()
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [140]:
# Run a minibatch of data through the model to check we get any errors
minibatch = train_x[:64]
model
model(train_x)

tensor([[ 0.0044],
        [ 0.0082],
        [-0.0114],
        ...,
        [-0.8352],
        [-0.3547],
        [-0.1872]], grad_fn=<AddmmBackward>)

In [141]:
def binary_acc(y_pred, y_test):
    # Transform outputs to 0 and 1
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    # Calculate percentage of correct predictions
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [142]:
# Specify that we want to use the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [143]:
# Move model to GPU if possible
model = model.to(device)
# Tells PyTorch we are in training mode
model.train()

for e in range(0, EPOCHS):
    
    # Set loss and accuracy to zero at start of each epoch
    epoch_training_loss = 0
    epoch_training_accuracy = 0
    epoch_valid_loss = 0
    epoch_valid_accuracy = 0
    
    for x_batch, y_batch in train_dl:
        # Transfer the tensors to the GPU if possible
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        
        # Zero out gradients before backpropagation (PyTorch cumulates the gradient otherwise)
        optimizer.zero_grad()
        
        # Predict a minibatch of outputs
        y_pred = model(x_batch)
        
        # Calculate the loss (unsqueeze adds a dimension to y)
        loss = loss_function(y_pred, y_batch.unsqueeze(1))
        training_acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        # Backpropagation. Gradients are calculated
        loss.backward()
        optimizer.step()
        
        epoch_training_loss += loss.item()
        epoch_training_accuracy += training_acc.item()
    
    for x_batch, y_batch in valid_dl:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        
        valid_y_pred = model(x_batch)
        valid_loss = loss_function(valid_y_pred, y_batch.unsqueeze(1))
        valid_acc = binary_acc(valid_y_pred, y_batch.unsqueeze(1))
        
        epoch_valid_loss += valid_loss.item()
        epoch_valid_accuracy += valid_acc.item()
        
    avg_train_loss = epoch_training_loss/len(train_dl)
    avg_valid_loss = epoch_training_loss/len(valid_dl)
    
    avg_train_accuracy = epoch_training_accuracy/len(train_dl)
    avg_valid_accuracy = epoch_valid_accuracy/len(valid_dl)
    
    print(f'Epoch {e}: | Training Loss: {avg_train_loss:.5f} | Training accuracy: {avg_train_accuracy} | Validation Loss: {avg_valid_loss} | Validation Accuracy: {avg_valid_accuracy}')

Epoch 0: | Training Loss: 0.59955 | Training accuracy: 65.55147058823529 | Validation Loss: 2.3981900243683154 | Validation Accuracy: 68.32352941176471
Epoch 1: | Training Loss: 0.55346 | Training accuracy: 69.72058823529412 | Validation Loss: 2.213834304201936 | Validation Accuracy: 67.91176470588235
Epoch 2: | Training Loss: 0.52091 | Training accuracy: 72.23529411764706 | Validation Loss: 2.0836425790890627 | Validation Accuracy: 66.47058823529412
Epoch 3: | Training Loss: 0.48172 | Training accuracy: 75.375 | Validation Loss: 1.926896418934874 | Validation Accuracy: 67.5
Epoch 4: | Training Loss: 0.43179 | Training accuracy: 79.19117647058823 | Validation Loss: 1.7271532145975936 | Validation Accuracy: 65.73529411764706
Epoch 5: | Training Loss: 0.37222 | Training accuracy: 82.73529411764706 | Validation Loss: 1.4888631815595352 | Validation Accuracy: 62.94117647058823
Epoch 6: | Training Loss: 0.34809 | Training accuracy: 83.97058823529412 | Validation Loss: 1.3923772384140656 | V

In [144]:
y_pred_list = np.array([])

model.eval()

with torch.no_grad():
    for X_batch, y_batch in valid_dl:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_tag = y_pred_tag.squeeze(1).cpu().numpy()
        y_pred_list = np.append(y_pred_list, y_pred_tag)
                     
len(y_pred_list)

2172

In [145]:
!pip install -Uqq sklearn

In [146]:
from sklearn.metrics import confusion_matrix, classification_report 

In [147]:
confusion_matrix(valid_y, y_pred_list)

array([[1067,  371],
       [ 410,  324]])